In [1]:
import pandas as pd
import seoapikits

In [2]:
# https://cdn.dataforseo.com/v3/examples/python/python_Client.zip
keywords_list = pd.read_excel('./sample_data/await_finding_url.xlsx', engine='openpyxl')['company_name_cv'].to_list()[:5]

We read the sample data, we only pick the first 5, then Initialize the cls to search for

In [3]:
username = input("Enter the username: ")
password = input("Enter the password: ")
search_api_cls = seoapikits.SERP.SERPAPI(username=username, password=password)

After, we try to post the data, which means we send request to the seodata server, and we can get the task id, Then we could directly got the result by using get. However if request "get" so fast it could not finish search and cast error, so we can use timeout limit to make it to retry. default freq is 5 seconds and timeout is 0 seconds, However you can reset it to make better usage, here is the example

In [4]:
task_post_df = search_api_cls.tasks_seq_post(keyword_list=keywords_list,res_type='df')
task_post_df

,id,keyword
0,09201039-6469-0066-0000-d6358662f68f,"""Johnnie"" Johnson Housing Trust Ltd."
1,09201039-6469-0066-0000-a25cb8786a40,051 Subs
2,09201039-6469-0066-0000-53e6b66722f5,063 Loan Company Limited
3,09201039-6469-0066-0000-09ba2e9b805f,0880 Santander Santibañez Sl
4,09201039-6469-0066-0000-32e51f446bc5,0km Finance


In [5]:
task_get_df = search_api_cls.tasks_seq_get_df(task_post_df['id'], timeout_limit=0)
task_get_df

  0%|          | 0/5 [00:00<?, ?it/s]


ConnectionError: Error-timeout for not get a efficient response from Seodata

You can see, so fast search will result for not enough time for server to response. So we take time to retry.
More easily, you could use ```search_api_cls.task_ready``` to checkin

Wait for a while and check again. We could have a simple method which have a while to wait for all task finished

In [7]:
task_ready_df = search_api_cls.tasks_seq_ready_df(task_post_df['id'])
task_ready_df

,id,ready
0,09201039-6469-0066-0000-d6358662f68f,True
1,09201039-6469-0066-0000-a25cb8786a40,True
2,09201039-6469-0066-0000-53e6b66722f5,True
3,09201039-6469-0066-0000-09ba2e9b805f,True
4,09201039-6469-0066-0000-32e51f446bc5,True


In [8]:
task_get_df = search_api_cls.tasks_seq_get_df(task_post_df['id'], timeout_limit=30)

100%|██████████| 5/5 [00:05<00:00,  1.15s/it]


In [9]:
pd.merge(task_get_df, task_post_df, on='id', validate='many_to_one')

,type,rank_group,rank_absolute,domain,title,description,url,breadcrumb,id,keyword
0,organic,1,1,www.jjhousing.co.uk,Johnnie Johnson Housing: Housing Association,We are Johnnie Johnson Housing ... We are a no...,https://www.jjhousing.co.uk/,https://www.jjhousing.co.uk,09201039-6469-0066-0000-d6358662f68f,"""Johnnie"" Johnson Housing Trust Ltd."
1,organic,2,2,uk.linkedin.com,Johnnie Johnson Housing,Johnnie Johnson Housing is a not-for-profit ho...,https://uk.linkedin.com/company/johnniejohnson...,https://uk.linkedin.com › company › johnniejoh...,09201039-6469-0066-0000-d6358662f68f,"""Johnnie"" Johnson Housing Trust Ltd."
2,organic,3,4,www.findahomejjhousing.co.uk,Login - JJ Housing,Johnnie Johnson Housing Limited is a Registere...,https://www.findahomejjhousing.co.uk/login/,https://www.findahomejjhousing.co.uk › login,09201039-6469-0066-0000-d6358662f68f,"""Johnnie"" Johnson Housing Trust Ltd."
3,organic,4,5,www.facebook.com,Johnnie Johnson Housing | Stockport,"Johnnie Johnson Housing, Cheadle Hulme. 1268 l...",https://www.facebook.com/jjhousingtrust/,https://www.facebook.com › jjhousingtrust,09201039-6469-0066-0000-d6358662f68f,"""Johnnie"" Johnson Housing Trust Ltd."
4,organic,5,6,find-and-update.company-information.service.go...,"""johnnie"" johnson housing trust limited - Comp...","""JOHNNIE"" JOHNSON HOUSING TRUST LIMITED - Free...",https://find-and-update.company-information.se...,https://find-and-update.company-information.se...,09201039-6469-0066-0000-d6358662f68f,"""Johnnie"" Johnson Housing Trust Ltd."
...,...,...,...,...,...,...,...,...,...,...
370,organic,95,96,www.globalfleet.com,Uruguay,Bank financing facilities are better for 0km c...,https://www.globalfleet.com/en/wikifleet/uruguay,https://www.globalfleet.com › ... › Wikifleet ...,09201039-6469-0066-0000-32e51f446bc5,0km Finance
371,organic,96,97,www.faganmotorcycles.co.nz,0 - New and Used Kawasaki and Suzuki Motorcycl...,0km. Express Finance Calculator. Vehicle Cost....,https://www.faganmotorcycles.co.nz/motorcycle/...,https://www.faganmotorcycles.co.nz › motorcycl...,09201039-6469-0066-0000-32e51f446bc5,0km Finance
372,organic,97,98,blog.clutch.ca,12 Cheapest New Cars in Canada in 2023,"... km warranty. Plus, you get a 10-day or 750...",https://blog.clutch.ca/posts/6-cheapest-new-ca...,https://blog.clutch.ca › posts › 6-cheapest-ne...,09201039-6469-0066-0000-32e51f446bc5,0km Finance
373,organic,98,99,www.porsche.com,Porsche Financial Services,Seguro de até 4 anos diluído nas parcelas de f...,https://www.porsche.com/brazil/pt/accessoriesa...,https://www.porsche.com › brazil,09201039-6469-0066-0000-32e51f446bc5,0km Finance
